In [1]:
import pandas as pd
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

df = pd.read_json('/content/drive/MyDrive/Colab_Notebooks/Entity Recognition in Resumes.json', lines=True)
df.head()


Mounted at /content/drive


,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [2]:
df = df.drop(['extras'], axis=1)
df.head()

,content,annotation
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20..."


In [3]:
!pip install pytorch-pretrained-bert
import numpy as np
import pandas as pd

import spacy
from spacy.training import offsets_to_biluo_tags
import en_core_web_sm

nlp = en_core_web_sm.load()

from tqdm import trange
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2ForTokenClassification, AdamW




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [4]:
pip install --upgrade spacy


In [5]:
!pip install --no-deps seqeval[gpu]
from seqeval.metrics import classification_report, accuracy_score, f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=576d4970d833ffb5eeaa87e73bf5d77c3bf1bc36a86be4dd2be77b3ed08727a9
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_json('/content/drive/MyDrive/Colab_Notebooks/Entity Recognition in Resumes.json', lines=True)
# df.head()


In [7]:
# Adding '\n' to the default spacy tokenizer

prefixes = ('\\n',) + tuple(nlp.Defaults.prefixes)

prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [8]:
entity_dict = {
    'Name': 'NAME',
    'College Name': 'CLG',
    'Degree': 'DEG',
    'Graduation Year': 'GRADYEAR',
    'Years of Experience': 'YOE',
    'Companies worked at': 'COMPANY',
    'Designation': 'DESIG',
    'Skills': 'SKILLS',
    'Location': 'LOC',
    'Email Address': 'EMAIL'
}

In [9]:
# # Since, 'extras' column contains no information we can drop the column
# df = df.drop(['extras'], axis=1)

In [10]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [11]:
def get_entities(df):

    entities = []

    for i in range(len(df)):
        entity = []

        for annot in df['annotation'][i]:
            try:
                ent = entity_dict[annot['label'][0]]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass

        entity = mergeIntervals(entity)
        entities.append(entity)

    return entities

In [12]:
# Adding a new column 'entities'
df['entities'] = get_entities(df)

In [13]:
def get_train_data(df):
    tags = []
    sentences = []

    for i in range(len(df)):
        text = df['content'][i]
        entities = df['entities'][i]

        doc = nlp(text)

        tag = offsets_to_biluo_tags(doc, entities)
        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text == '.' and tmp[1][i] == 'O':
                loc.append(i)
        loc.append(len(doc))

        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos

        for d in data:
            tag = ['O' if t == '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)

    return sentences, tags

In [14]:
sentences, tags = get_train_data(df)
len(sentences), len(tags)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with entities "[(0, 14, 'NAME'), (62, 68, 'LOC'), (104, 148, 'EMA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Akhil Yadav Polemaina
Hyderabad, Telangana - Email..." with entities "[(0, 21, 'NAME'), (22, 31, 'LOC'), (65, 117, 'EMAI...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

(780, 780)

In [15]:
tag_vals = set(['X', '[CLS]', '[SEP]'])
for i in range(len(tags)):
    tag_vals = tag_vals.union(tags[i])
tag_vals

{'B-CLG',
 'B-COMPANY',
 'B-DEG',
 'B-DESIG',
 'B-EMAIL',
 'B-GRADYEAR',
 'B-LOC',
 'B-NAME',
 'B-SKILLS',
 'B-YOE',
 'I-CLG',
 'I-COMPANY',
 'I-DEG',
 'I-DESIG',
 'I-EMAIL',
 'I-GRADYEAR',
 'I-LOC',
 'I-NAME',
 'I-SKILLS',
 'I-YOE',
 'L-CLG',
 'L-COMPANY',
 'L-DEG',
 'L-DESIG',
 'L-EMAIL',
 'L-GRADYEAR',
 'L-LOC',
 'L-NAME',
 'L-SKILLS',
 'L-YOE',
 'O',
 'U-CLG',
 'U-COMPANY',
 'U-DEG',
 'U-DESIG',
 'U-EMAIL',
 'U-GRADYEAR',
 'U-LOC',
 'U-SKILLS',
 'U-YOE',
 'X',
 '[CLS]',
 '[SEP]'}

In [16]:
tag2idx = {t: i for i, t in enumerate(tag_vals)}
tag2idx

{'U-SKILLS': 0,
 'L-DESIG': 1,
 'U-EMAIL': 2,
 'I-GRADYEAR': 3,
 'B-LOC': 4,
 'L-GRADYEAR': 5,
 '[CLS]': 6,
 'U-YOE': 7,
 'I-CLG': 8,
 'O': 9,
 'I-DESIG': 10,
 'U-DESIG': 11,
 '[SEP]': 12,
 'I-EMAIL': 13,
 'B-SKILLS': 14,
 'L-DEG': 15,
 'U-GRADYEAR': 16,
 'I-SKILLS': 17,
 'B-COMPANY': 18,
 'I-NAME': 19,
 'B-DESIG': 20,
 'B-EMAIL': 21,
 'B-YOE': 22,
 'B-NAME': 23,
 'B-GRADYEAR': 24,
 'I-LOC': 25,
 'L-YOE': 26,
 'L-NAME': 27,
 'U-COMPANY': 28,
 'I-YOE': 29,
 'B-DEG': 30,
 'U-CLG': 31,
 'B-CLG': 32,
 'U-LOC': 33,
 'L-LOC': 34,
 'X': 35,
 'L-EMAIL': 36,
 'L-COMPANY': 37,
 'I-COMPANY': 38,
 'L-SKILLS': 39,
 'L-CLG': 40,
 'U-DEG': 41,
 'I-DEG': 42}

In [17]:
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}
idx2tag

{0: 'U-SKILLS',
 1: 'L-DESIG',
 2: 'U-EMAIL',
 3: 'I-GRADYEAR',
 4: 'B-LOC',
 5: 'L-GRADYEAR',
 6: '[CLS]',
 7: 'U-YOE',
 8: 'I-CLG',
 9: 'O',
 10: 'I-DESIG',
 11: 'U-DESIG',
 12: '[SEP]',
 13: 'I-EMAIL',
 14: 'B-SKILLS',
 15: 'L-DEG',
 16: 'U-GRADYEAR',
 17: 'I-SKILLS',
 18: 'B-COMPANY',
 19: 'I-NAME',
 20: 'B-DESIG',
 21: 'B-EMAIL',
 22: 'B-YOE',
 23: 'B-NAME',
 24: 'B-GRADYEAR',
 25: 'I-LOC',
 26: 'L-YOE',
 27: 'L-NAME',
 28: 'U-COMPANY',
 29: 'I-YOE',
 30: 'B-DEG',
 31: 'U-CLG',
 32: 'B-CLG',
 33: 'U-LOC',
 34: 'L-LOC',
 35: 'X',
 36: 'L-EMAIL',
 37: 'L-COMPANY',
 38: 'I-COMPANY',
 39: 'L-SKILLS',
 40: 'L-CLG',
 41: 'U-DEG',
 42: 'I-DEG'}

In [18]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForTokenClassification.from_pretrained(model_name, num_labels=len(tag2idx))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import AdamW
def get_tokenized_train_data(sentences, tags):

    tokenized_texts = []
    word_piece_labels = []

    for word_list, label in zip(sentences, tags):

        # Add [CLS] at the front
        temp_lable = ['[CLS]']
        temp_token = ['[CLS]']

        for word, lab in zip(word_list, label):
            token_list = tokenizer.tokenize(word.text)
            for m, token in enumerate(token_list):
                temp_token.append(token)
                if m == 0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')

        # Add [SEP] at the end
        temp_lable.append('[SEP]')
        temp_token.append('[SEP]')

        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)

    return tokenized_texts, word_piece_labels

In [20]:
tokenized_texts, word_piece_labels = get_tokenized_train_data(sentences, tags)

In [21]:
print(tokenized_texts[0])
print(word_piece_labels[0])

['[CLS]', 'Ab', 'his', 'he', 'k', 'J', 'ha', 'Ċ', 'Application', 'Development', 'Ass', 'ociate', '-', 'Acc', 'enture', 'Ċ', 'Ċ', 'B', 'eng', 'al', 'uru', ',', 'K', 'arn', 'ataka', '-', 'Email', 'me', 'on', 'Indeed', ':', 'ind', 'eed', '.', 'com', '/', 'r', '/', 'Ab', 'his', 'he', 'k', '-', 'J', 'ha', '/', '10', 'e', '7', 'a', '8', 'cb', '7', '32', 'bc', '43', 'a', 'Ċ', 'Ċ', 'âĢ¢', 'To', 'work', 'for', 'an', 'organ', 'ization', 'which', 'prov', 'ides', 'me', 'the', 'opp', 'ortun', 'ity', 'to', 'improve', 'my', 'sk', 'ills', 'Ċ', 'and', 'knowledge', 'for', 'my', 'individual', 'and', 'company', "'s", 'growth', 'in', 'best', 'p', 'ossible', 'ways', '[SEP]']
['[CLS]', 'B-NAME', 'X', 'X', 'X', 'L-NAME', 'X', 'O', 'B-DESIG', 'I-DESIG', 'L-DESIG', 'X', 'O', 'U-COMPANY', 'X', 'O', 'O', 'U-LOC', 'X', 'X', 'X', 'O', 'O', 'X', 'X', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 

In [22]:
MAX_LEN = 512
bs = 4

In [23]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post", dtype="long", truncating="post")
attention_masks = [[float(i > 0) for i in ii] for ii in input_ids]

In [24]:
from sklearn.utils.class_weight import compute_class_weight
# Compute class weights
flat_tags = [item for sublist in tags for item in sublist]
class_weights = compute_class_weight('balanced', classes=np.unique(flat_tags), y=flat_tags)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [25]:
tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks = train_test_split(input_ids, tags, attention_masks, random_state=2020,
                                                                                 test_size=0.3)

In [26]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [27]:
# Create DataLoader
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)



In [28]:
# Optimizer setup
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'ln_1', 'ln_2']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
from transformers import  get_linear_schedule_with_warmup
epochs = 10
# Scheduler
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


In [ ]:
# Ensure class weights are on the same device as labels
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = class_weights.to(device)

# Training
epochs = 10
max_grad_norm = 1.0

model.to(device)

for epoch in range(epochs):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        weighted_loss = loss * class_weights[b_labels].mean()
        weighted_loss.backward()
        tr_loss += weighted_loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    print(f"Epoch {epoch} ===== Train loss: {tr_loss / nb_tr_steps}")

Epoch 0 ===== Train loss: 1.2092200263614092
Epoch 1 ===== Train loss: 0.19230894236587479
Epoch 2 ===== Train loss: 0.15284951385382536
Epoch 3 ===== Train loss: 0.12350247617484662
Epoch 4 ===== Train loss: 0.1099145183081392
Epoch 5 ===== Train loss: 0.10245610507422664
Epoch 6 ===== Train loss: 0.09491160974232819
Epoch 7 ===== Train loss: 0.0873739547300132


In [ ]:
# Evaluation
model.eval()
y_true = []
y_pred = []
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=input_mask)
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()
    for i, mask in enumerate(input_mask):
        temp_1 = []
        temp_2 = []
        for j, m in enumerate(mask):
            if m:
                temp_1.append(idx2tag[label_ids[i][j]])
                temp_2.append(idx2tag[np.argmax(logits[i][j])])
            else:
                break
        y_true.append(temp_1)
        y_pred.append(temp_2)

print("F1 score: %f" % (f1_score(y_true, y_pred)))
print("Accuracy score: %f" % (accuracy_score(y_true, y_pred)))
print(classification_report(y_true, y_pred, digits=4))